# Part 2 of interpretability of a Convnet model


In this case the idea now is to plot what the filters see when they activate at their highest level

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
import math

/var/folders/8s/wcgky8714x9115n0kp3x7ysm0000gn/T/ipykernel_5865/2554390667.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# load the 3 models we already've trained

In [2]:
# model1 = tf.keras.models.load_model("./checkpoints/model1/")

# model2 = tf.keras.models.load_model("./checkpoints/model2/")

# model3 = tf.keras.models.load_model("./checkpoints/model3/")

model3 = tf.keras.models.load_model("./checkpoints/esp32/")

Metal device set to: Apple M1 Max

systemMemory: 64.00 GB
maxCacheSize: 24.00 GB



2023-05-25 21:14:17.710111: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-25 21:14:17.710581: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# First step build a loss function

The idea is to maximize the activation

In [3]:
def loss(activation):
    """
        initial_random_image: initial random image made to max out the activation of the filter.
        prediction: output of the filter's activation.
        Squared Error

        # Trying to maximize the activations average.
    """

    return tf.reduce_mean(activation)

In [4]:
def preproces_img(image_output):

    filter_patern = image_output.copy()
    
    filter_patern -= filter_patern.mean()
    filter_patern /= filter_patern.std()
    filter_patern *= 64
    filter_patern += 128
    filter_patern = np.clip(filter_patern, 0, 255).astype("uint8")

    #Center image
    return filter_patern[25:-25, 25:-25, :]

In [5]:
def get_convs_layers(model):
    outputs=[]
    for layer in model.layers:
        if isinstance(layer, (tf.keras.layers.Conv2D)):
            layer.trainable=False
            outputs.append(layer)
    return outputs

In [6]:
def print_filters(model_name, layer_name, all_images, image_shape=(256, 256)):       
    margin = 5
    # all_images = [preproces_img(image) for image in generated_image_from_filters]

    # print(len(all_images))

    c = 4
    r = math.ceil(len(all_images)/c)

    # print(c, r)

    cropped_width = image_shape[0]-25*2
    cropped_height = image_shape[1]-25*2

    width = r * cropped_width + (r-1) * margin
    height = c * cropped_height + (c-1) * margin

    stitched_filters = np.zeros((width, height, 3))

    # print(width, height)

    #rows
    for i in tqdm(range(r)):
        for j in range(c):
            # print(i*c+j)
            image = all_images[i * c +j]

            row_start= (cropped_width + margin) * i
            row_end = (cropped_height + margin) * i + cropped_width
            column_start = (cropped_height + margin) * j
            column_end = (cropped_height + margin) * j + cropped_height

            # print("row_start, row_end, column_start, column_end")
            # print(row_start, row_end, column_start, column_end)

            stitched_filters[row_start: row_end, column_start: column_end, :] = image

    tf.keras.utils.save_img(f"./part2-filters/{model_name}_{layer_name}.png", stitched_filters)

In [7]:
@tf.function
def optimize_step(feature_extractor_model, image, filter_index, lr):
    with tf.GradientTape() as tape:
        tape.watch(image)
        activation = feature_extractor_model(image)
        activation = activation[:, 2:-2, 2:-2, filter_index]
        loss_value = loss(activation)
        
    gradients = tape.gradient(loss_value, image)
    gradients = tf.math.l2_normalize(gradients)
    image += gradients*lr
    return image, loss_value

In [8]:
model3.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 96, 96, 1)]       0         
                                                                 
 rescaling (Rescaling)       (None, 96, 96, 1)         0         
                                                                 
 reshape_1 (Reshape)         (None, 96, 96, 1)         0         
                                                                 
 1st_conv_3x3 (Conv2D)       (None, 94, 94, 4)         40        
                                                                 
 1st_max_pooling (MaxPooling  (None, 47, 47, 4)        0         
 2D)                                                             
                                                                 
 2nd_conv_3x3 (Conv2D)       (None, 45, 45, 8)         296       
                                                             

In [10]:
model = model3
image_shape = (256, 256)
filter_index = 0
sample_layer = model.get_layer("2nd_conv_3x3")
feature_extractor_model = tf.keras.Model(inputs=model.input, outputs=sample_layer.output)
lr=10

In [11]:
loss_value = 0
while loss_value == 0:
    noise_image = tf.random.uniform(minval=0.0, maxval=1.0, shape=(1000,)+image_shape+(3,))
    loss_value = loss(feature_extractor_model(noise_image)[:, :, :, 0])

ValueError: Input 0 of layer "model" is incompatible with the layer: expected shape=(None, 96, 96, 1), found shape=(1000, 256, 256, 3)

In [ ]:
index = tf.where(tf.reduce_mean(feature_extractor_model(noise_image)[:, :, :, 0], axis=[1, 2], keepdims=False)).numpy()[0][0]

NameError: name 'feature_extractor_model' is not defined

In [10]:
loss(feature_extractor_model(tf.expand_dims(noise_image[index], axis=0))[:, :, :, 0])

NameError: name 'feature_extractor_model' is not defined

In [11]:
del noise_image

NameError: name 'noise_image' is not defined

In [12]:
# %%time
# image_shape = (256, 256)

# # models=[model1, model2, model3]
# # models_name = ["model_1_new_version", "model_2_new_version", "model_3_new_version"]

# models=[model3]
# models_name = ["model_3_new_version"]

# epochs=100
# learning_rate = 10

# for model_name, model in zip(models_name, models):
#     for layer in get_convs_layers(model):
#         feature_extractor_model = tf.keras.Model(inputs=model.input, outputs=layer.output)
#         images = []
#         filters_count = layer.output.shape[-1]
#         print(f"layer: {layer.name}")
#         for filter_index in range(filters_count):
#             print(f"filter: index{filter_index}")
#             noise_image = tf.random.uniform(minval=0.4, maxval=0.6, shape=(1,)+image_shape+(3,))
#             for epoch in range(epochs):
#                 noise_image = optimize_step(feature_extractor_model, noise_image, filter_index, learning_rate)
#             images.append(preproces_img(noise_image.numpy()[0].copy()))
#         print_filters(model_name=model_name, layer_name=layer.name, all_images=images)

In [13]:
130%5==0

True

In [14]:
10**-1

0.1

In [15]:
exponent=3
for cicles in range(50):
    exponent *= np.exp((cicles-15)/25)
    exponent = math.ceil(exponent)
    print(exponent)
    print(10**(-exponent))

2
0.01
2
0.01
2
0.01
2
0.01
2
0.01
2
0.01
2
0.01
2
0.01
2
0.01
2
0.01
2
0.01
2
0.01
2
0.01
2
0.01
2
0.01
2
0.01
3
0.001
4
0.0001
5
1e-05
6
1e-06
8
1e-08
11
1e-11
15
1e-15
21
1e-21
31
1e-31
47
1e-47
73
1e-73
118
1e-118
199
1e-199
349
0.0
636
0.0
1207
0.0
2383
0.0
4896
0.0
10470
0.0
23302
0.0
53976
0.0
130131
0.0
326537
0.0
852816
0.0
2318195
0.0
6558677
0.0
19313203
0.0
59192152
0.0
188819016
0.0
626901211
0.0
2166328266
0.0
7791502301
0.0
29166876275
0.0
113639787977
0.0


In [17]:
%%time
image_shape = (96, 96)

# models=[model1, model2, model3]
# models_name = ["model_1_new_version", "model_2_new_version", "model_3_new_version"]

models=[model3]
models_name = ["model_3_Alternative_noise_generation_improved"]

epochs=10
learning_rate = 10

for model_name, model in tqdm(zip(models_name, models), total=1):
    for layer in get_convs_layers(model):
        feature_extractor_model = tf.keras.Model(inputs=model.input, outputs=layer.output)
        images = []
        filters_count = layer.output.shape[-1]
        
        print(f"layer: {layer.name}")
        for filter_index in tqdm(range(filters_count), leave=False):
            print(f"filter index: {filter_index+1}")
            exponent = 3
            cicles = 0
            # noise_image = tf.random.uniform(minval=0.0, maxval=1.0, shape=(1,)+image_shape+(3,))

            #generate an image that will trigger the filter's response
            print("generating initial image to trigger loss")
            loss_target_value = 0
            while loss_target_value < 10**(-exponent):
                print(f"target value: {10**(-exponent)}")
                noise_image_options = tf.random.uniform(minval=0.0, maxval=1.0, shape=(1000,)+image_shape+(1,))
                loss_target_value = loss(feature_extractor_model(noise_image_options)[:, :, :, filter_index])
                
                cicles+=1
                print(f"cicles: {cicles}")
                if cicles%100==0:
                    exponent += 1
                    print(f"Increase exponent by 1 now it's: {exponent}")

                exponent = math.ceil(exponent)
                print(f"exponent: {exponent}")
            
            print(f"loss init value: {loss_target_value}")
            non_zero_loss_index = tf.where(tf.reduce_mean(feature_extractor_model(noise_image_options)[:, :, :, filter_index], axis=[1, 2], keepdims=False)).numpy()[0][0]

            print(f"random image index: {non_zero_loss_index}")

            noise_image = tf.expand_dims(noise_image_options[non_zero_loss_index], axis=0)

            if loss(feature_extractor_model(noise_image)[:, :, :, filter_index]) == 0:
                print(f"Loss zero for {filter_index}")
                # assert 1!=0
            
            print("Optimization step!")
            for epoch in range(epochs*(exponent)):
                noise_image = optimize_step(feature_extractor_model, noise_image, filter_index, learning_rate)

            images.append(preproces_img(noise_image.numpy()[0].copy()))

        print_filters(model_name=model_name, layer_name=layer.name, all_images=images)

  0%|          | 0/1 [00:00<?, ?it/s]

layer: 1st_conv_3x3


filter index: 1
generating initial image to trigger loss
target value: 0.001
cicles: 1
exponent: 3
loss init value: 0.03727852180600166
random image index: 0


Optimization step!
filter index: 2
generating initial image to trigger loss
target value: 0.001
cicles: 1
exponent: 3
loss init value: 0.028986509889364243
random image index: 0
Optimization step!


filter index: 3
generating initial image to trigger loss
target value: 0.001
cicles: 1
exponent: 3
loss init value: 0.0654105618596077
random image index: 0
Optimization step!
filter index: 4
generating initial image to trigger loss
target value: 0.001
cicles: 1
exponent: 3
target value: 0.001
cicles: 2
exponent: 3
target value: 0.001
cicles: 3
exponent: 3
target value: 0.001
cicles: 4
exponent: 3
target value: 0.001
cicles: 5
exponent: 3
target value: 0.001
cicles: 6
exponent: 3
target value: 0.001
cicles: 7
exponent: 3
target value: 0.001
cicles: 8
exponent: 3
target value: 0.001
cicles: 9
exponent: 3
target value: 0.001
cicles: 10
exponent: 3
target value: 0.001
cicles: 11
exponent: 3
target value: 0.001
cicles: 12
exponent: 3
target value: 0.001
cicles: 13
exponent: 3
target value: 0.001
cicles: 14
exponent: 3
target value: 0.001
cicles: 15
exponent: 3
target value: 0.001
cicles: 16
exponent: 3
target value: 0.001
cicles: 17
exponent: 3
target value: 0.001
cicles: 18
exponent: 3
ta

  0%|          | 0/1 [00:26<?, ?it/s]

cicles: 3473
exponent: 37
target value: 1e-37
cicles: 3474
exponent: 37
target value: 1e-37
cicles: 3475
exponent: 37
target value: 1e-37
cicles: 3476
exponent: 37
target value: 1e-37
cicles: 3477
exponent: 37
target value: 1e-37
cicles: 3478
exponent: 37
target value: 1e-37
cicles: 3479
exponent: 37
target value: 1e-37
cicles: 3480
exponent: 37
target value: 1e-37
cicles: 3481
exponent: 37
target value: 1e-37
cicles: 3482
exponent: 37
target value: 1e-37
cicles: 3483
exponent: 37
target value: 1e-37
cicles: 3484
exponent: 37
target value: 1e-37
cicles: 3485
exponent: 37
target value: 1e-37
cicles: 3486
exponent: 37
target value: 1e-37
cicles: 3487
exponent: 37
target value: 1e-37
cicles: 3488
exponent: 37
target value: 1e-37
cicles: 3489
exponent: 37
target value: 1e-37
cicles: 3490
exponent: 37
target value: 1e-37
cicles: 3491
exponent: 37
target value: 1e-37
cicles: 3492
exponent: 37
target value: 1e-37
cicles: 3493
exponent: 37
target value: 1e-37
cicles: 3494
exponent: 37
target v

IndexError: index 0 is out of bounds for axis 0 with size 0